In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np

In [ ]:
#generate ground truth and drift measurement
mean = 4.0
std_dev = 0.2
length = 1000
drift_free_array = torch.normal(mean=mean, std=std_dev, size=(length,))


def create_sinusoidal_drift(length, amplitude=0.5, frequency=1.0):
    t = torch.arange(length)
    drift = amplitude * torch.sin(2 * np.pi * frequency * t / length)
    return drift

amplitude = 0.5
frequency = 1.0

num_measurements = 1000
patch_length = 20


drifted_measurements = []
ground_truth = []


for _ in range(num_measurements):
    start_idx = torch.randint(0, length - patch_length + 1, (1,)).item()
    
    patch = drift_free_array[start_idx:start_idx + patch_length]
    
    drift = create_sinusoidal_drift(patch_length, amplitude, frequency)
    
    drifted_patch = patch + drift
    
    drifted_measurements.append(drifted_patch)
    ground_truth.append(patch)

drifted_measurements_tensor = torch.stack(drifted_measurements)
ground_truth_tensor = torch.stack(ground_truth)

dataset = TensorDataset(drifted_measurements_tensor, ground_truth_tensor)

train_ratio = 0.8 

train_length = int(train_ratio * num_measurements)
val_length = num_measurements - train_length

train_dataset, val_dataset = random_split(dataset, [train_length, val_length])

batch_size = 32 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU()
        )
        self.decoder_drifted = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(),
            nn.Linear(10, 20)
        )
        self.decoder_drift_free = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(),
            nn.Linear(10, 20)
        )
    
    def forward(self, x):
        latent = self.encoder(x)
        drifted_output = self.decoder_drifted(latent)
        drift_free_output = self.decoder_drift_free(latent)
        return drifted_output, drift_free_output

In [ ]:
autoencoder = Autoencoder()
mse_loss = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [ ]:
num_epochs = 100 

for epoch in range(num_epochs):
    total_loss = 0.0
    total_val_loss = 0.0
    
    autoencoder.train()
    for drifted_batch, ground_truth_batch in train_loader:
        drifted_output, drift_free_output = autoencoder(drifted_batch)

        drifted_loss = mse_loss(drifted_output, drifted_batch)
        drift_free_loss = mse_loss(drift_free_output, ground_truth_batch)

        loss = drifted_loss + drift_free_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    autoencoder.eval()
    with torch.no_grad():
        for drifted_batch, ground_truth_batch in val_loader:
            drifted_output, drift_free_output = autoencoder(drifted_batch)

            drifted_loss = mse_loss(drifted_output, drifted_batch)
            drift_free_loss = mse_loss(drift_free_output, ground_truth_batch)

            loss = drifted_loss + drift_free_loss

            total_val_loss += loss.item()
    
    avg_train_loss = total_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

Epoch 1/100, Train Loss: 33.09877899169922, Validation Loss: 32.589642660958425
Epoch 2/100, Train Loss: 31.77648338317871, Validation Loss: 30.397002356392996
Epoch 3/100, Train Loss: 27.315767669677733, Validation Loss: 22.591331481933594
Epoch 4/100, Train Loss: 16.604866828918457, Validation Loss: 10.954116140093122
Epoch 5/100, Train Loss: 7.925512065887451, Validation Loss: 5.232173102242606
Epoch 6/100, Train Loss: 3.6235390377044676, Validation Loss: 2.264804090772356
Epoch 7/100, Train Loss: 1.4687604093551636, Validation Loss: 0.8512161459241595
Epoch 8/100, Train Loss: 0.5365648078918457, Validation Loss: 0.3191154769488743
Epoch 9/100, Train Loss: 0.21817026495933534, Validation Loss: 0.15420228668621608
Epoch 10/100, Train Loss: 0.12135739505290985, Validation Loss: 0.10280043312481471
Epoch 11/100, Train Loss: 0.09042074978351593, Validation Loss: 0.08505382069519588
Epoch 12/100, Train Loss: 0.08014327824115754, Validation Loss: 0.07920873271567481
Epoch 13/100, Train Lo